In [1]:
import SimpleITK as sitk
import ciclope
import meshio
import ciclope.utils
from ciclope.utils.preprocess import remove_unconnected
import scipy
import numpy as np
import mcubes
import pygalmesh
from ciclope.utils.recon_utils import read_tiff_stack
from skimage.filters import threshold_otsu, gaussian
from scipy import ndimage, misc

import ciclope.utils.preprocess

In [ ]:
input_file = '/Users/gianthk/Code/ORMIR/ciclope/test_data/LHDL/3155_D_4_bc/cropped/3155_D_4_bc_0000.tif'
samplename = 'trab' # 'scaffold'
output_dir = '/Users/gianthk/Desktop/'

In [ ]:
# read test image
data_3D = read_tiff_stack(input_file)
voxelsize = 19.5e-3
vs = np.ones(3)*voxelsize # [mm]

In [ ]:
# gaussian filter
data_3D = gaussian(data_3D, sigma=1, preserve_range=True)

In [ ]:
# downsample
resampling = 1

# resize the 3D data using spline interpolation of order 2
# data_3D = ndimage.zoom(data_3D, 1/resampling, output=None, order=2)

# correct voxelsize
vs = vs * resampling

In [ ]:
# thresholding
T = threshold_otsu(data_3D)
# bw = data_3D > T
bw = data_3D > 63 # from comparison with histology

# remove unconnected components
bw = remove_unconnected(bw)

# simplify
# bw = scipy.ndimage.binary_closing(bw,iterations=1, border_value=1)
# bw = scipy.ndimage.binary_opening(bw,iterations=1, border_value=0)

# add endplates
# bw = ciclope.utils.preprocess.add_cap(np.transpose(bw, axes=(1,2,0)), 10, 1)

In [ ]:
# direct generation of tetrahedra mesh from array
filename_mesh_out = output_dir+samplename+'_tetramesh.vtk'
mesh_size_factor = 2
m1 = ciclope.tetraFE.cgal_mesh(bw, vs, 'tetra', mesh_size_factor*min(vs), 2*mesh_size_factor*min(vs))

In [ ]:
# write mesh to file
m1.write(filename_mesh_out)

In [ ]:
# pad binary image with a layer of zeros
bw = np.pad(bw, 3, 'constant', constant_values=0)

In [ ]:
# smooth the surface of binary image
# bw = mcubes.smooth(bw)

In [ ]:
# generate surface mesh using marching cubes
# https://github.com/pmneila/PyMCubes/tree/master

vertices, triangles = mcubes.marching_cubes(bw, 0.5)

vertices = vertices * voxelsize
vertices = vertices[:,(2,1,0)]

In [ ]:
# write surface mesh
filename_surfacemesh_out = output_dir+samplename+'_surfacemesh.obj'
mcubes.export_obj(vertices, triangles, filename_surfacemesh_out)

In [ ]:
# remeshing an existing surface mesh
# as described here: https://github.com/meshpro/pygalmesh?tab=readme-ov-file#surface-remeshing

# m11 = pygalmesh.remesh_surface(
#     filename_surfacemesh_out,
#     max_edge_size_at_feature_edges=3*voxelsize,
#     min_facet_angle=20,
#     max_radius_surface_delaunay_ball=5*voxelsize,
#     max_facet_distance=3*voxelsize,
#     verbose=True,
# )

In [ ]:
# write mesh to file

# filename_mesh_out = output_dir+samplename+'_surfacemesh_refined.vtk'
# m11.write(filename_mesh_out)

In [ ]:
# volume mesh from surface mesh
m2 = pygalmesh.generate_volume_mesh_from_surface_mesh(
    filename_surfacemesh_out,
    min_facet_angle=20.0,
    max_radius_surface_delaunay_ball=4*voxelsize,
    max_facet_distance=voxelsize,
    max_circumradius_edge_ratio=3.0,
    verbose=True,
    reorient=True,
)

In [ ]:
# write mesh to file
filename_mesh_out = output_dir+samplename+'_tetramesh_from_surface.vtk'
m2.write(filename_mesh_out)